In [2]:
import pickle
import numpy as np

In [3]:
def mnist_load():
    with open("mnist.pkl", "rb") as f:
        mnist = pickle.load(f)
    return (
        mnist["training_images"][:len(mnist["training_images"])//2]/255.0,
        [[int(i == y) for i in range(10)] for y in mnist["training_labels"][:len(mnist["training_labels"])//2]],
        mnist["test_images"][:len(mnist["test_images"])//2]/255.0,
        [[int(i == y) for i in range(10)] for y in mnist["test_labels"][:len(mnist["test_labels"])//2]],
    )

In [4]:
def get_distance(x, y):
    return np.linalg.norm(x - y, axis=0)

In [5]:
def kmeans_cust(xs, k, max_iters=1000):
    conv = False
    centroids = xs[np.random.choice(range(len(xs)), size=k, replace=False)]
    
    it = 0
    while (not conv) and it < max_iters:
        clusters = [[] for _ in range(len(centroids))]
        for x in xs:
            min_dist = np.inf
            closest_c = -1
            for j, c in enumerate(centroids):
                dist = get_distance(x, c)
                if dist < min_dist:
                    min_dist = dist
                    closest_c = j
            clusters[closest_c].append(x)
        clusters = [c for c in clusters if c]
        prev_centroids = centroids.copy()
        centroids = [np.mean(c, axis=0) for c in clusters]
        
        conv = len(prev_centroids) == len(centroids) and np.allclose(prev_centroids, centroids)
        it += 1
            
    return np.array(centroids), [np.std(x) for x in clusters]

In [6]:
from sklearn.cluster import KMeans

In [7]:
def kmeans(xs, k, max_iters=1000):
    centroids = KMeans(n_clusters=k, max_iter=max_iters).fit(xs).cluster_centers_

    max_d = -1
    for i in range(len(centroids)):
        for j in range(i, len(centroids)):
            max_d = max(get_distance(centroids[i], centroids[j]), max_d)
    return np.array(centroids), [max_d for c in centroids]

In [8]:
class RBF:
  def __init__(self, k, size):
    self.classes = 10
    self.k = k
    self.size = size
    self.centroids = None
    self.stds = None
    self.weights = None
  def rbf(self, x, c, std):
    return 1 / np.exp(-get_distance(x, c) / std**2)
  def train(self, X, y):
    self.centroids, self.stds = kmeans(X, self.k)
    
    act = np.array([np.array([self.rbf(x, c, s) for c, s in zip(self.centroids, self.stds)]) for x in X])

    self.weights = np.linalg.pinv(act.T @ act) @ act.T @ y

    out = act @ self.weights
    cnt = 0
    for i in range(X.shape[0]):
      if np.argmax(out[i]) == np.argmax(y[i]):
        cnt += 1
    
    return cnt / X.shape[0]

  

In [9]:
train_data, train_labels, test_data, test_labels = mnist_load()
print(len(test_data))

5000


In [10]:
ks = [10, 40, 80, 100]
for k in ks:
  rbf = RBF(k, 10)
  print(f'Clusters: {k}, Accuracy: {rbf.train(train_data, train_labels)}')


Clusters: 10, Accuracy: 0.6293666666666666
Clusters: 40, Accuracy: 0.8625
Clusters: 80, Accuracy: 0.8975666666666666
Clusters: 100, Accuracy: 0.9068333333333334
